<a href="https://colab.research.google.com/github/ag4267research1/Solving-Poisson-s-Equation-with-GNO/blob/main/GNOPDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install neuraloperator --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 47.5 MB/s eta 0:00:00


In [5]:
import os, pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

# Correct imports:
from neuralop.layers.gno_block import GNOBlock
from neuralop.data.transforms.normalizers import UnitGaussianNormalizer
from neuralop.training import Trainer



In [ ]:
# Folder in the Colab VM (not Drive)
!mkdir -p /content/nonlinear_poisson
!cd /content/nonlinear_poisson

# Download with progress bar (percentage + speed)
!wget --show-progress \
  "https://zenodo.org/records/15001788/files/nonlinear_poisson.obj?download=1" \
  -O nonlinear_poisson.obj




--2025-12-02 18:56:39--  https://zenodo.org/records/15001788/files/nonlinear_poisson.obj?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.75, 188.185.43.153, 137.138.52.235, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9608410281 (8.9G) [application/octet-stream]
Saving to: ‘nonlinear_poisson.obj’

nonlinear_poisson.o   6%[>                   ] 556.56M  1.59MB/s    eta 73m 17s

In [ ]:
# Show file size
!ls -lh nonlinear_poisson.obj

# MODEL

In [6]:
class GNOPoissonModel(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, coord_dim=2,
                 hidden_channels=64, radius=0.2, n_layers=3):
        super().__init__()

        layers = []
        c_in = in_channels
        for _ in range(n_layers):
            layers.append(
                GNOBlock(
                    in_channels=c_in,
                    out_channels=hidden_channels,
                    coord_dim=coord_dim,
                    radius=radius,
                    # keep defaults: linear transform_type, sum reduction, transformer PE, etc.
                )
            )
            c_in = hidden_channels

        self.gno_layers = nn.ModuleList(layers)
        self.out_layer = nn.Linear(hidden_channels, out_channels)

    def forward(self, x, coords):
        """
        x:      (B, P, C_in)
        coords: (B, P, coord_dim)
        """
        h = x
        for gno in self.gno_layers:
            h = gno(h, coords)
            h = F.gelu(h)

        # pointwise readout at each node
        out = self.out_layer(h)
        return out